In [1]:
import napari
import napari_workflows as nw
from skimage.io import imread
file_name = r'C:/Users/ryans/Desktop/blobs.tif'
blobs = imread(filename)

viewer = napari.Viewer()
viewer.add_image(blobs)

C:\Users\ryans\anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'blobs' at 0x166cd3fc2b0>

In [3]:
wf_manager = nw.WorkflowManager.install(viewer)

In [4]:
wf = wf_manager.workflow
print(wf)

Workflow:
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x00000166FE1615A0>, 'blobs', 5.0)
Result of Threshold (Ridler et al 1978, scikit-image, nsbatwm) <- (<function threshold_mean at 0x00000166FE1611B0>, 'Result of Gaussian (scikit-image, nsbatwm)')



In [5]:
from napari_workflows._io_yaml_v1 import load_workflow, save_workflow
        
filename = 'workflow from napari.yaml'

save_workflow(filename, wf)

In [7]:
loaded_wf = load_workflow(filename)
print(loaded_wf)

Workflow:
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x00000166FE1615A0>, 'blobs', 5.0)
Result of Threshold (Ridler et al 1978, scikit-image, nsbatwm) <- (<function threshold_mean at 0x00000166FE1611B0>, 'Result of Gaussian (scikit-image, nsbatwm)')



In [10]:
import inspect
import magicgui
from functools import partial
from napari_tools_menu import make_gui

viewer2 = napari.Viewer()

from inspect import Signature, signature
from functools import partial

def signature_w_kwargs_from_function(function, arg_vals: list) -> Signature:
    
    # getting the keywords corresponding to the values
    keyword_list = list(signature(function).parameters.keys())

    # creating the kwargs dict
    kw_dict = {}
    for kw, val in zip(keyword_list, arg_vals):
        kw_dict[kw] = val
    kw_dict.pop('image') # we are making an assumption that the input will aways be this
    
    sig = signature(partial(function, **kw_dict))
    
    return sig




for task in loaded_wf._tasks.keys():
    func = loaded_wf._tasks[task][0]  # extract function from workflow
    args = loaded_wf._tasks[task][1:] # extract args from wf
    
    signat = signature_w_kwargs_from_function(func,args)
    func.__signature__ = signat
    
    widget = make_gui(func, viewer2)
    
    viewer2.window.add_dock_widget(widget)

C:\Users\ryans\anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [11]:
roots = loaded_wf.roots()
roots

['blobs']

In [12]:
loaded_wf._tasks.items()

dict_items([('Result of Gaussian (scikit-image, nsbatwm)', (<function gaussian_blur at 0x00000166FE1615A0>, 'blobs', 5.0)), ('Result of Threshold (Ridler et al 1978, scikit-image, nsbatwm)', (<function threshold_mean at 0x00000166FE1611B0>, 'Result of Gaussian (scikit-image, nsbatwm)'))])

In [14]:
def wf_steps_with_root_as_input(workflow):
    roots = loaded_wf.roots()
    wf_step_with_rootinput = []
    for result, task in workflow._tasks.items():
            for source in task:
                if isinstance(source, str):
                    if source in roots:
                        wf_step_with_rootinput.append(result)
    return wf_step_with_rootinput

wf_steps_with_root_as_input(loaded_wf)             

['Result of Gaussian (scikit-image, nsbatwm)']